## Player Stats
Collect statistics for teams at a player level. Player stats are buried in the JSON respone by game and player id. A function must be used to iteratively pull data from the responses making for a slightly more complicated process to wrangle player data. Here the individual stats per game are collected for all teams and saved. There will be a notebook which aggregates each stat type and combines them into a final play stats csv file.

In [1]:
import requests
import pandas as pd
pd.options.mode.chained_assignment = None

In [31]:
#Get all team names
r = requests.post('https://profootballapi.com/teams?api_key=9SLf6UbjIWVYevtA5Tmi1HuphNK7Gr08&year=2009')
team = r.json()
team_names = (pd.DataFrame(team))['team'].unique()
team_names

array(['TEN', 'BUF', 'PIT', 'ARI', 'PHI', 'NE', 'BAL', 'WAS', 'OAK',
       'DAL', 'NYJ', 'STL', 'IND', 'MIN', 'NO', 'CIN', 'SF', 'DEN', 'DET',
       'ATL', 'CHI', 'GB', 'CLE', 'KC', 'HOU', 'TB', 'SD', 'SEA', 'NYG',
       'CAR', 'MIA', 'JAC'], dtype=object)

In [53]:
#utility for extracting player stats per team and appending lists, abstracted for clarity
#Had to be done by team as constrained by API to limit data sent per request

api_key = '9SLf6UbjIWVYevtA5Tmi1HuphNK7Gr08'

rushing = []
passing = []
recieving = []
defense = []

def add_player_stats(team_name):
    """ Given a team name it appends stats per game per player for 
    the categories of rushing, recieving, and passing"""
    
    r1 = requests.post('https://profootballapi.com/players?api_key=' + api_key + '&season_type=REG&stats_type=offense&team=' + team_name)
    team = r1.json()
    
    r2 = requests.post('https://profootballapi.com/players?api_key=' + api_key + '&season_type=REG&stats_type=defense&team=' + team_name)
    def_ = r2.json()
    
    for game in def_.keys():
        for player in def_[game].keys():
            if type(def_[game][player]) != type({}):
                continue
            if def_[game][player].get('defense') is not None:
                d = def_[game][player].get('defense')
                d['game_id'] = int(game)
                d['team'] = team_name
                defense.append(d)
    
    for game in team.keys():
        for player in team[game].keys():
            if type(team[game][player]) != type({}):
                continue
            if team[game][player].get('rushing') is not None:
                rush = team[game][player].get('rushing')
                rush['game_id'] = int(game)
                rush['team'] = team_name
                rushing.append(rush)
            if team[game][player].get('passing') is not None:
                pass_ = team[game][player].get('passing')
                pass_['game_id'] = int(game)
                pass_['team'] = team_name
                passing.append(pass_)
            if team[game][player].get('receiving') is not None:
                recieve = team[game][player].get('receiving')
                recieve['game_id'] = int(game)
                recieve['team'] = team_name
                recieving.append(recieve)
    return


In [54]:
#Collect the data for players
for name in team_names:
    add_player_stats(name)

In [55]:
r = pd.DataFrame(rushing)
r.head()

,attempts,game_id,long,long_touchdown,name,team,touchdowns,two_point_attempts,two_point_makes,yards
0,2,2009091000,1,0,K.Collins,TEN,0,0,0,1
1,15,2009091000,32,0,C.Johnson,TEN,0,0,0,57
2,8,2009091000,6,0,L.White,TEN,0,0,0,28
3,3,2009092007,3,0,K.Collins,TEN,0,0,0,4
4,16,2009092007,91,0,C.Johnson,TEN,2,0,0,197


In [56]:
p = pd.DataFrame(passing)
p.head()

,attempts,completions,game_id,interceptions,name,team,touchdowns,two_point_attempts,two_point_makes,yards
0,35,22,2009091000,1,K.Collins,TEN,1,0,0,244
1,33,21,2009092007,1,K.Collins,TEN,2,0,0,216
2,37,15,2009092711,2,K.Collins,TEN,1,0,0,170
3,48,29,2009100404,2,K.Collins,TEN,1,1,0,284
4,32,19,2009101112,1,K.Collins,TEN,0,0,0,164


In [57]:
c = pd.DataFrame(recieving)
c.head()

,game_id,long,long_touchdown,name,receptions,team,touchdowns,two_point_attempts,two_point_makes,yards
0,2009091000,11,0,C.Johnson,1,TEN,0,0,0,11
1,2009091000,5,0,L.White,1,TEN,0,0,0,5
2,2009091000,16,0,J.Gage,7,TEN,1,0,0,78
3,2009091000,20,0,B.Scaife,5,TEN,0,0,0,48
4,2009091000,57,0,K.Britt,4,TEN,0,0,0,85


In [58]:
d = pd.DataFrame(defense)
d.head()

,assisted_tackles,forced_fumbles,game_id,interceptions,name,sacks,tackles,team
0,1,0,2009091000,1,C.Finnegan,0,9,TEN
1,4,0,2009091000,0,S.Tulloch,0,8,TEN
2,4,0,2009091000,0,C.Hope,0,7,TEN
3,5,0,2009091000,0,N.Harper,0,4,TEN
4,2,1,2009091000,0,M.Griffin,0,4,TEN


In [40]:
#Get games df to merge and obtain yearly information
games = pd.read_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\games.csv')[['id','year']]
games.head()

,id,year
0,2009091000,2009
1,2009091300,2009
2,2009091301,2009
3,2009091302,2009
4,2009091303,2009


In [70]:
#Merge for year, rushing stats
r_clean = games.merge(r, left_on='id', right_on='game_id')[['year','game_id','team','name','attempts','touchdowns','yards']]
r_clean.head()

,year,game_id,team,name,attempts,touchdowns,yards
0,2009,2009091000,TEN,K.Collins,2,0,1
1,2009,2009091000,TEN,C.Johnson,15,0,57
2,2009,2009091000,TEN,L.White,8,0,28
3,2009,2009091000,PIT,B.Roethlisberger,1,0,3
4,2009,2009091000,PIT,W.Parker,13,0,19


In [69]:
#Merge for year, passing stats
p_clean = games.merge(p, left_on='id', right_on='game_id')[['year','game_id','team','name','attempts','completions','interceptions','touchdowns','yards']]
p_clean.head()

,year,game_id,team,name,attempts,completions,interceptions,touchdowns,yards
0,2009,2009091000,TEN,K.Collins,35,22,1,1,244
1,2009,2009091000,PIT,B.Roethlisberger,43,33,2,1,363
2,2009,2009091300,ATL,M.Ryan,36,22,0,2,229
3,2009,2009091300,MIA,C.Pennington,29,21,1,1,176
4,2009,2009091300,MIA,P.White,1,0,0,0,0


In [68]:
#Merge for year, recieving stats
c_clean = games.merge(c, left_on='id', right_on='game_id')[['year','game_id','team','name','long','receptions','touchdowns','yards']]
c_clean.head()

,year,game_id,team,name,long,receptions,touchdowns,yards
0,2009,2009091000,TEN,C.Johnson,11,1,0,11
1,2009,2009091000,TEN,L.White,5,1,0,5
2,2009,2009091000,TEN,J.Gage,16,7,1,78
3,2009,2009091000,TEN,B.Scaife,20,5,0,48
4,2009,2009091000,TEN,K.Britt,57,4,0,85


In [67]:
#Merge for year, defensive stats
d_clean = games.merge(d, left_on='id', right_on='game_id')[['year','game_id','team','name','sacks','tackles','forced_fumbles','interceptions']]
d_clean.head()

,year,game_id,team,name,sacks,tackles,forced_fumbles,interceptions
0,2009,2009091000,TEN,C.Finnegan,0,9,0,1
1,2009,2009091000,TEN,S.Tulloch,0,8,0,0
2,2009,2009091000,TEN,C.Hope,0,7,0,0
3,2009,2009091000,TEN,N.Harper,0,4,0,0
4,2009,2009091000,TEN,M.Griffin,0,4,1,0


In [71]:
#Save to csv
r_clean.to_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\rushing.csv',index=False)
p_clean.to_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\passing.csv',index=False)
c_clean.to_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\recieving.csv',index=False)
d_clean.to_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\defensive.csv',index=False)